# Read data

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('房产_链家二手房_北京_2014-2021.csv')
df.head(5)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df = df = df.rename(columns={'Transaction_Time）':'Transaction_Time'})

In [ ]:
df.columns

# Explore data

In [ ]:
# check null value
df.isnull().sum()

In [ ]:
#filling missing values with mean
df['Transaction_Cycle(day)'] = df['Transaction_Cycle(day)'].fillna(df['Transaction_Cycle(day)'].mean())
df['Average_Price'] = df['Average_Price'].fillna(df['Average_Price'].mean())
df['Building_Year'] = df['Building_Year'].fillna(df['Building_Year'].mean())

In [ ]:
#transform data
df['Building_Year'] = df['Building_Year'].astype(np.int64)
df['Transaction_Cycle(day)'] = df['Transaction_Cycle(day)'].astype(np.int64)
df['Average_Price'] = df['Average_Price'].astype(np.int64)
df['Transaction_Time'] = pd.to_datetime(df['Transaction_Time'])

In [ ]:
#check date information again
df.info()

In [ ]:
# encode categorical fields
df['Floor'].unique()

In [ ]:
df = df.dropna(subset=['Floor','Building_Type','Building_structure'])

In [ ]:
df['Floor'].unique()

In [ ]:
df['Building_Type'].unique()

In [ ]:
df['Renovation'].unique()

In [ ]:
df['Elevator'].unique()

In [ ]:
df['Duty_Free'].unique()

In [ ]:
df['Subway_or_not'].unique()

In [ ]:
df['District'].unique()

In [ ]:
df.isnull().sum()

In [ ]:
df['Floor'].replace(['底层', '底楼层', '顶层', '高楼层', '中楼层'],['Bottom','Low','Middle','High','Top'],inplace=True)

In [ ]:
df['Building_Type'].replace(['板楼', '板塔结合', '塔楼', '平房'],['Slab_building','Slab_building_mixed_tower','Building_like_tower','Bungalow'],inplace=True)

In [ ]:
df['Renovation'].replace(['简装', '精装', '其他', '毛坯'],['Fine_decoration','Delicate_decoration','Others','No_decoration'],inplace=True)

In [ ]:
df['Building_structure'].replace(['混合结构', '钢混结构', '砖混结构', '砖木结构', '钢结构'],['Mixed_structure','Steel-concrete_structure','Brick_structure','Brick_and_wood_structure','Steel_structure'],inplace=True)

In [ ]:
df['Elevator'].replace(['无', '有'],['Yes','No'],inplace=True)

In [ ]:
df['Duty_Free'].replace('满五年','Yes',inplace=True)

In [ ]:
df['Subway_or_not'].replace(['否', '是'],['No','Yes'],inplace=True)

In [ ]:
df['District'].replace(['石景山', '海淀', '昌平', '西城', '顺义', '朝阳', '丰台', '东城', '通州', '平谷', '大兴',
       '门头沟','房山'],['SJ','HD','CP','XC','SY','CY','FT','DC','TZ','PG','DX','MTG','FS'],inplace=True)

In [ ]:
#Remove data with wrong date(Select data by time range)
s_date = '2014-01-01'
e_date = '2021-12-31'
df = df[(df['Transaction_Time'] >= s_date) & (df['Transaction_Time'] <= e_date)]

In [ ]:
# Save data to csv before encoding
#df.to_csv('C:/Python/Practice5005/df_clean1.csv')

In [ ]:
# Encoding
df_1 = df
df_1['Floor'].replace(['Bottom','Low','Middle','High','Top'],[0,1,2,3,4],inplace=True)
df_1['Building_Type'].replace(['Slab_building','Slab_building_mixed_tower','Building_like_tower','Bungalow'],[3,2,1,0],inplace=True)
df_1['Renovation'].replace(['Fine_decoration','Delicate_decoration','Others','No_decoration'],[2,3,1,0],inplace=True)
df_1['Building_structure'].replace(['Mixed_structure','Steel-concrete_structure','Brick_structure','Brick_and_wood_structure','Steel_structure'],[2,3,1,0,4],inplace=True)
df_1['Elevator'].replace(['Yes','No'],[1,0],inplace=True)
df_1['Subway_or_not'].replace(['No','Yes'],[0,1],inplace=True)
df_1.head()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
dis_price = df_1.groupby('District')['Average_Price'].mean()
dis_price

In [ ]:
type(dis_price)

In [ ]:
dis = pd.DataFrame(dis_price, columns=['Average_Price'])
dis.index

In [ ]:
plt.bar(x =dis.index ,height=dis['Average_Price'])
plt.title('Average house price by different regions')
plt.show()

In [ ]:
df_1['District'].replace(['SJ','HD','CP','XC','SY','CY','FT','DC','TZ','PG','DX','MTG','FS'],[7,10,3,12,2,9,8,11,6,4,5,1,0],inplace=True)

In [ ]:
df_1.head(3)

In [ ]:
# Save data to csv after encoding
#df_1.to_csv('C:/Python/Practice5005/df_clean2.csv')

In [ ]:
df_1.info()

# Factor Analysis

In [ ]:
#extract features
df_fa = df_1.drop(columns = ['ID','Duty_Free','Transaction_Time','Price_PerSquare','BedRoom','SittingRoom','Bathroom','Kitchen','Average_Price','Building_Height'])
df_fa.dropna()
df_fa.shape

In [ ]:
df_fa.info()

In [ ]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
chi_square_value,p_value=calculate_bartlett_sphericity(df_fa)
chi_square_value, p_value

In [ ]:
from factor_analyzer.factor_analyzer import calculate_kmo
kmo_all,kmo_model=calculate_kmo(df_fa)
print(kmo_model)

In [ ]:
from factor_analyzer import FactorAnalyzer
fa = FactorAnalyzer(df_fa.shape[1]+1, rotation=None)
fa.fit(df_fa)
ev, v = fa.get_eigenvalues()

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(range(1,df_fa.shape[1]+1),ev)
plt.plot(range(1,df_fa.shape[1]+1),ev)
plt.title('scree plot',fontdict={'weight':'normal','size': 25})
plt.xlabel('Fa_score.csvactor',fontdict={'weight':'normal','size': 15})
plt.ylabel('Eigenvalues',fontdict={'weight':'normal','size': 15})
plt.grid()
plt.show()

In [ ]:
n_factors = sum(ev>=1)
n_factors

In [ ]:
#Take the rotated result
fa2 = FactorAnalyzer(n_factors,rotation='varimax',method='principal')
fa2.fit(df_fa)
#Give the contribution rate
var = fa2.get_factor_variance()
var

In [ ]:
fa2.loadings_

In [ ]:
import seaborn as sns
df_fa_loading = pd.DataFrame(fa2.loadings_,index=df_fa.columns)

In [ ]:
fig,ax = plt.subplots(figsize=(8,8))
sns.heatmap(df_fa_loading,annot=True,cmap='BuPu',ax=ax)
ax.tick_params(axis='x',labelsize=15)
ax.set_title("Factor Analysis",fontsize=12)
ax.set_ylabel("columns")

In [ ]:
df_fa_loading.shape

In [ ]:
fa2_score = fa2.transform(df_fa)

In [ ]:
column_list = ['fac'+str(i) for i in np.arange(n_factors)+1]
fa_score_df = pd.DataFrame(fa2_score,columns=column_list)
for col in fa_score_df.columns:
    df_fa[col] = fa_score_df[col]
fa_score_df

# DBSCAN Clustering

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
n_samples, n_features = fa2_score.shape
feature_names = fa_score_df.columns

In [ ]:
# try model_1 by default para
model_1 = DBSCAN().fit(fa2_score)

In [ ]:
model_1.labels_

In [ ]:
plt.scatter(fa2_score[:,0],fa2_score[:,1],c=model_1.labels_)
plt.show()

In [ ]:
# select better para(eps,min_samples)
res=[]
lit=[500,1000,2000,3000]

for Eps in np.arange(0.65,0.85,0.05):
    for Min_samples in lit:
        model = DBSCAN(eps=Eps,min_samples=Min_samples).fit(fa2_score)
        n_clusters = len([i for i in set(model.labels_) if i != -1])
        outliners = np.sum(np.where(model.labels_ == -1))
        res.append({'eps':Eps,'min_samples':Min_samples,'n_clusters':n_clusters,'outliners':outliners})
df_res = pd.DataFrame(res)
df_res

In [ ]:
#build model based on eps=0.5,min_samples=1000
model_2 = DBSCAN(eps=0.5,min_samples=1000).fit(fa2_score)

In [ ]:
print(len(model_2.labels_))
print(model_2.labels_)

In [ ]:
plt.scatter(fa2_score[:,2],fa2_score[:,3],c=model_2.labels_)
plt.show()

In [ ]:
#calculate the numbers of different clusters
from collections import Counter
clusters = Counter(model_2.labels_)
print(clusters)

In [ ]:
colors = model_2.labels_
plt.scatter(df_geo['Longitude'],df_geo['Latitude'],c=colors)
plt.xlabel('Longitude',family = 'Arial',fontsize=9)
plt.ylabel('Latitude',family = 'Arial',fontsize=9)
plt.title('Clusters of houses in Beijing based on DBSCAN',family = 'Arial',fontsize=15)
plt.grid(which='major',color='#cccccc',alpha=0.45)
plt.show()

In [ ]:
# convert model labels（array format) into dataframe
df_label = pd.DataFrame(model_2.labels_)
df_label

In [ ]:
df_1['label'] = df_label[0]
df_1.head(3)

In [ ]:
df_1.columns

In [ ]:
#check features of the different clusters
df_1.groupby('label')['Transaction_Cycle(day)','Followers','Price_PerSquare','Area','BedRoom','SittingRoom','Kitchen', 'Bathroom', 'Floor',
       'Building_Height', 'Building_Type', 'Building_Year', 'Renovation',
       'Building_structure', 'Elevator_Ratio', 'Elevator', 'Duty_Free',
       'Subway_or_not', 'District', 'Average_Price'].mean()

# K-means clustering

In [ ]:
# Try different group numbers for clustering
from sklearn.cluster import KMeans

for i in range(3,7):
    km = KMeans(n_clusters=i).fit(fa2_score)
    rs_labels = km.labels_
    label_series=pd.Series(rs_labels)
    print(" %s clusters and counts are:" % i)
    print(label_series.value_counts())
    # print("Cluster centers are:")
    # print(km.cluster_centers_)

In [ ]:
# when k is equal to 5, the features of cluster_1 are similar to high-end apartments.
km_2 = KMeans(n_clusters=5).fit(fa2_score)

In [ ]:
df_1['label_1']=km_2.labels_
df_1.head(3)

In [ ]:
df_1.groupby('label_1')['Transaction_Time',
       'Transaction_Cycle(day)', 'Followers', 'Total_Price', 'Price_PerSquare',
       'Area', 'BedRoom', 'SittingRoom', 'Kitchen', 'Bathroom', 'Floor',
       'Building_Height', 'Building_Type', 'Building_Year', 'Renovation',
       'Building_structure', 'Elevator_Ratio', 'Elevator', 'Duty_Free',
       'Subway_or_not', 'District', 'Average_Price',].mean()

In [ ]:
type(df_1['label_1'])

In [ ]:
#df_1.to_csv('C:/Python/Practice5005/df_cluster.csv')

# DBSCAN Clustering based on location('Longitude', 'Latitude')

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 14,9  #set plot size

In [ ]:
df_geo = pd.read_csv('C:/Python/practice5005/df_cluster.csv',usecols=['ID','Longitude','Latitude','label_1'])

In [ ]:
df_high = df_geo.loc[df_geo['label_1'] == 1]
print(df_high.head())
print(len(df_high))

In [ ]:
#plot the geographical points
_ = plt.plot(df_high['Longitude'],df_high['Latitude'],marker='.',linewidth=0,color='#128128')
_ = plt.grid(which='major',color='#cccccc',alpha=0.45)
_ = plt.title('Geographical distribution of high-end houses in Beijing',family='Arial',fontsize=12)
_ = plt.xlabel('Longitude')
_ = plt.ylabel('Latitude')
plt.show()

In [ ]:
df_cluster = df_high[['Longitude','Latitude']]
df_cluster = df_cluster.values.astype('float32',copy=False)
df_cluster

In [ ]:
#normaliza data
from sklearn.preprocessing import StandardScaler
model_scaler = StandardScaler().fit(df_cluster)
df_cluster_scaler = model_scaler.transform(df_cluster)
df_cluster_scaler

In [ ]:
res_1=[]
lit_1=[500,1000,2000,3000]

for Eps in np.arange(0.25,0.45,0.05):
    for Min_samples in lit_1:
        model = DBSCAN(eps=Eps,min_samples=Min_samples).fit(df_cluster_scaler)
        n_clusters = len([i for i in set(model.labels_) if i != -1])
        outliners = np.sum(np.where(model.labels_ == -1))
        res_1.append({'eps':Eps,'min_samples':Min_samples,'n_clusters':n_clusters,'outliners':outliners})
df_res_1 = pd.DataFrame(res_1)
df_res_1

In [ ]:
#build model based on normaliza data
model_3 = DBSCAN(eps=0.25,min_samples=1000).fit(df_cluster_scaler)
model_3

In [ ]:
from collections import Counter
clusters_3 = Counter(model_3.labels_)
clusters_3 

In [ ]:
colors3 = model_3.labels_
plt.scatter(df_high['Longitude'],df_high['Latitude'],c=colors3)
plt.xlabel('Longitude',family = 'Arial',fontsize=9)
plt.ylabel('Latitude',family = 'Arial',fontsize=9)
plt.title('Clusters of houses in Beijing based on DBSCAN',family = 'Arial',fontsize=15)
plt.grid(which='major',color='#cccccc',alpha=0.45)
plt.show()

In [ ]:
df_high['label_2']=model_3.labels_
df_high.head(3)

In [ ]:
#save df_high to csv
df_high.to_csv('C:/Python/Practice5005/df_high.csv')

In [ ]:
#plot in map
#pip install folium

In [ ]:
pip install geopy

In [ ]:
from geopy.geocoders import ArcGIS

In [ ]:
Beijian = ArcGIS().geocode('beijing')
Beijian 

In [ ]:
import folium
bj_map = folium.Map(location=[39.90750000000003, 116.39723000000004],zoom_start=8)
bj_map

In [ ]:
a_list = df_high[['ID','Longitude','Latitude','label_1']].values.tolist()
a_list

In [ ]:
import folium
bj_map1 = folium.Map(location=[39.98317261,116.4134249])
fg = folium.FeatureGroup(name='Clusters of houses in Beijing based on DBSCAN in the map')

for i in a_list:
    fg.add_child(folium.Marker(location=[i[2],i[1]],popup=i[3],icon=folium.Icon(color=i[3])))
bj_map1.add_child(fg)

In [ ]:
# extract data in 2014
#s_2014 = '2014-01-01'
#e_2015 = '2015-12-31'
#df_1415 = df[(df['Transaction_Time'] >= s_2014) & (df['Transaction_Time'] <= e_2015)]

In [ ]:
#normalized data
#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#df_cluster=scaler.fit_transform(df_fa)
#df_cluster

In [ ]:
#After normalization, the data categories become array
#type(df_cluster)